# Overview

Make sure that this notebook is run in the Conda environment setup for this project, as described in [Readme.md](README.md)

This notebook runs the code for performing schooling experiments. 

# Parameters

In [1]:
# Prefix for video files
moviefile_prefix = "SCHOOL"

# Address for the Enttec USB device (see above)
hw_address = "/dev/cu.usbserial-EN373474"

# Define IP address of smart switch (see README for how to find at command line)
LED_IP = "192.168.0.104"

# Whether to control the lights and recorder. Set to False when not connected to hardware, or when troubleshooting the code.
control_hw = True

# Packages

In [2]:

import numpy as np
import def_runexperiments as re
import multiprocess, os
# import pandas as pd

if control_hw:
        # For DMX Control
        from DMXEnttecPro import Controller

        # This is necessary for using multiprocess on a Mac
        if os.name == "posix":
                multiprocess.set_start_method('spawn')

## Project paths

You can find instructions for defining hw_address in [README.md](README.md). You will need to be connected to the fileserver ("vortex") to access the log_path.

In [3]:
if control_hw:
    # Path to time-code audio file
    aud_path = "/Users/experimentalist/Documents/Projects/wake_tracking/timecode.wav"

    # Check address of audio file
    if not os.path.isfile(aud_path):
        raise OSError("aud_path not found at " + aud_path)    
else:
    aud_path = None

# Path to log csv file
log_path = "/Volumes/schooling/TRex/SwimTest/recording_log.csv"
# log_path = "/Users/mmchenry/Documents/Projects/waketracking/SwimTest/recording_log.csv"

# Path to directory containing experiment schedules
schedules_path = "/Volumes/schooling/TRex/SwimTest/experiment_schedules"
# schedules_path = "/Users/mmchenry/Documents/Projects/waketracking/SwimTest/experiment_schedules"

# Check address of log file & scheudle directory
if not os.path.isfile(log_path):
    raise OSError("log_path not found at " + log_path)
if not os.path.isdir(schedules_path):
    raise OSError("schedules_path not found at " + schedule_path)

# log = pd.read_csv(log_path)
# print(log)

## Initialize hardware

Run these lines for all of the programs that you want to run below, even if you are not connected to the hardware.

In [4]:
if control_hw:
    #  Initialize hardware
    dmx = Controller(hw_address, auto_submit=True, dmx_size=32)

    # Set output rate at maximum
    dmx.set_dmx_parameters(output_rate=0)
    
else:
    dmx = None

# Basic hardware control

## Change fixed light intensity
Note that if the Luxli LED lamp becomes unresponsive, then disconnect the DMX connection at the Enttec USB box, then power off the power supply to the light, turn the power supply back on, then power up the light (it should show the interactive screen), and then reconnect the DMX.

In [ ]:
# Specify light intensity (from 0 to 1)
light_intensity = 1

# Sets DMX channel 1 to max 255 (Channel 1 is the intensity)
dmx.set_channel(1, int(light_intensity*255))  

## Turn on LED array

In [ ]:
os.system('kasa --host ' + LED_IP + ' on')

## Turn off LED array

In [ ]:
os.system('kasa --host ' + LED_IP + ' off')

## Trigger a recording

In [ ]:
rec_dur_sec = 3

import multiprocess
import time
from playsound import playsound

p = multiprocess.Process(target=playsound, args=(aud_path, ))
print('Starting audio to trigger video recording')
p.start()

start_time = time.time()
curr_time = 0

# Send data to dmx in loop until time runs out
while curr_time<rec_dur_sec:          

    # Total time elapsed since the timer started
    curr_time = time.time() - start_time

    # Briefly pause the code to keep from overloading the hardware
    time.sleep(0.1)

p.terminate()
print('Timecode audio ended.')

# Single-experiment programs

Select which of the cells below that make sense for the experiment that you want to run.

## Lights on for fixed duration

In [ ]:
# What light level to return to after the program
end_light_level = 0.5

# list of light levels
light_level = np.array([1])

# Duration that each level is fixed (s)
light_dur = np.array([3])

# Run control program
re.run_program(dmx, aud_path=aud_path, log_path=log_path, light_level=light_level, 
    light_dur=light_dur, trig_video=True, echo=False, plot_data=True, 
    movie_prefix=moviefile_prefix, LED_IP=LED_IP, control_hw=control_hw)

if control_hw:
    # Set ending log intensity
    dmx.set_channel(1, int(end_light_level*255))  

# print(df)

## Lights on -> Lights off

Ramp down light intensity at a constant rate.

In [ ]:
# list of light levels
light_level = np.array([1, 0])

# Duration that each level is fixed (s)
light_dur = np.array([10, 10])

# Duration for the changes in light level
ramp_dur = np.array([0.5])

# Play control levels into Enttec DMX
re.run_program(dmx, aud_path=aud_path, log_path=log_path, light_level=light_level, 
    light_dur=light_dur, ramp_dur=ramp_dur, trig_video=True, echo=False, plot_data=True,
    movie_prefix=moviefile_prefix, LED_IP=LED_IP, control_hw=control_hw)

# print(df)

## Lights off -> Lights on

Ramp down light intensity at a constant rate.

In [ ]:
# list of light levels
light_level = np.array([0, 1])

# Duration that each level is fixed
light_dur = np.array([5, 5])

# Duration for the changes in light level
ramp_dur = np.array([2])

# Play control levels into Enttec DMX
re.run_program(dmx, aud_path=aud_path, log_path=log_path, light_level=light_level, 
    light_dur=light_dur, ramp_dur=ramp_dur, trig_video=True, echo=False, plot_data=True,
    movie_prefix=moviefile_prefix, LED_IP=LED_IP, control_hw=control_hw)

# print(df)

## Ramp lights down and then up

Changing light intensity at a constant rate.

In [ ]:
# list of light levels
light_level = np.array([1, 0, 0.5])

# Duration that each level is fixed
light_dur = np.array([1, 2, 1])

# Duration for the changes in light level
ramp_dur = np.array([0.5, 0.5])

# Play control levels into Enttec DMX
re.run_program(dmx, aud_path=aud_path, log_path=log_path, light_level=light_level, 
    light_dur=light_dur, ramp_dur=ramp_dur, trig_video=True, echo=False, plot_data=True,
     movie_prefix=moviefile_prefix, LED_IP=LED_IP, control_hw=control_hw)

# print(df)

# Sequence of experiments

Using re.make_schedule, set change_var to 'ramp_dur_sec', 'start_dur_min', or 'end_dur_min' to vary an individual aspect of a lighting experiment. Then, set num_expts, min_val and max_val to the range of values for that variable among the experiments. Once the schedule file has been created, then use re.run_experiment_schedule to run the experiments. Note that the schedule is specific to a particular date and that the experiments should be run on that particular date. it is therefore easiest to create the schedule file on the same day as the running of the experiments.

## Make schedule for experiments that vary ramp duration

A similar structure to the following code could be used where change_var is set to 'start_dur_min', or 'end_dur_min'.

In [5]:
# Delay before starting experiment (min)
start_delay_min = 10/60

# Duration of ramp before and after experiment (s)
pre_ramp_dur_sec = 3
post_ramp_dur_sec = 3

# Duration that each level is fixed at the start and end of experiment (min)
start_dur_min = 1
end_dur_min   = 1

# list of starting and ending light levels
light_start = 1
light_end   = 0

# Light intensity between experiments
light_btwn = 0.5

# Variable to change across experiments
change_var = 'ramp_dur_sec'

# Duration for the ramp of changes in light level (s)
min_rampdur_sec = 1
max_rampdur_sec = 10

# Number of experiments to run
num_expts = 5

# Period of time bwteen expts (min)
btwn_period_min = 0.5

# Write schedule to file
sch_file = re.make_schedule(schedules_path, change_var=change_var,  light_start=light_start, 
                            light_end=light_end, light_btwn=light_btwn, start_dur_min=start_dur_min, end_dur_min=end_dur_min, 
                            min_val=min_rampdur_sec, max_val=max_rampdur_sec, num_val=num_expts, btwn_period_min=btwn_period_min, pre_ramp_dur_sec=pre_ramp_dur_sec, post_ramp_dur_sec=post_ramp_dur_sec, start_delay_min=start_delay_min)

--------------------------------------------------
Schedule file created: /Volumes/schooling/TRex/SwimTest/experiment_schedules/2023-02-27_sch3.csv
--------------------------------------------------
 


## Execute experiments on a schedule

In [6]:
# Set this path to the schedule file you want to run
# sch_file = '/Users/mmchenry/Documents/Projects/waketracking/SwimTest/experiment_schedules/2023-02-27_sch2.csv'
sch_file = '/Volumes/schooling/TRex/SwimTest/experiment_schedules/2023-02-27_sch2.csv'

# Take number on video file for first recording
take_num_start = 1

# # Turn on the LEDs, wait for action to take
if control_hw and (LED_IP is not None):
    os.system('kasa --host ' + LED_IP + ' on')
    print('    Turning on LED array')
    time.sleep(3)

# Execute experiments from schedule
re.run_experiment_schedule(dmx, aud_path=aud_path, log_path=log_path, 
    schedule_path=sch_file, movie_prefix=moviefile_prefix, LED_IP=LED_IP, 
    control_hw=control_hw, take_num_start=take_num_start)

# Turn off the LEDs
if control_hw and (LED_IP is not None):
    os.system('kasa --host ' + LED_IP + ' off')
    print('    Turning off LED array')

Starting trial 3 at 2023-02-27, 14:18:29
    Starting audio to trigger video recording
    Timecode audio ended.
    Video WILL be analyzed
    Video filename: SCHOOL_S001_S001_T004
    Log file saved to: /Volumes/schooling/TRex/SwimTest/recording_log.csv
    Trial 3 complete!
Starting trial 4 at 2023-02-27, 18:20:03
    Starting audio to trigger video recording


KeyboardInterrupt: 